In [10]:
import os, cv2, numpy as np, pandas as pd, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from tqdm import tqdm
from sklearn.metrics import log_loss

device = torch.device('cuda')
label_cols = ['C1','C2','C3','C4','C5','C6','C7']

train_df = pd.read_csv('data/train_mips.csv')             # StudyInstanceUID + labels
y_df = pd.read_csv('train.csv')                           # StudyInstanceUID + C1..C7
bbox_df = pd.read_csv('train_bounding_boxes.csv')         # must include StudyInstanceUID, vertebra in C1..C7, and box columns
mip_root = 'data/mips/train'

# Debug: Print columns of bbox_df
print('Bounding boxes columns:', bbox_df.columns.tolist())
print('First few rows:'); print(bbox_df.head())

def normalize_boxes(df, img_size=384):
    df = df.copy()
    if all(c in df.columns for c in ['x1','y1','x2','y2']):
        print('Using x1,y1,x2,y2 format')
        pass
    elif all(c in df.columns for c in ['x','y','width','height']):
        print('Using x,y,width,height format')
        df['w'] = df['width']
        df['h'] = df['height']
        # assume pixels; if normalized, multiply by 384 here
        df['x1'] = df['x'] - df['w']/2.0
        df['y1'] = df['y'] - df['h']/2.0
        df['x2'] = df['x'] + df['w']/2.0
        df['y2'] = df['y'] + df['h']/2.0
    elif all(c in df.columns for c in ['x','y','w','h']):
        print('Using x,y,w,h format')
        # assume pixels; if normalized, multiply by 384 here
        df['x1'] = df['x'] - df['w']/2.0
        df['y1'] = df['y'] - df['h']/2.0
        df['x2'] = df['x'] + df['w']/2.0
        df['y2'] = df['y'] + df['h']/2.0
    else:
        print('Available columns:', df.columns.tolist())
        raise ValueError('train_bounding_boxes.csv must have x1,y1,x2,y2 or x,y,width,height or x,y,w,h')
    for c in ['x1','x2','y1','y2']: df[c] = df[c].clip(0, img_size-1)
    return df

bbox_df = normalize_boxes(bbox_df)

def make_patch_index(train_df, y_df, bbox_df, img_size=384, expand=1.5, min_side=96, target=224):
    lbl = train_df[['StudyInstanceUID']].merge(
        y_df[['StudyInstanceUID'] + label_cols], on='StudyInstanceUID', how='left'
    ).set_index('StudyInstanceUID')
    rows = []
    for uid, g in bbox_df.groupby('StudyInstanceUID'):
        if uid not in lbl.index:
            continue
        y7 = lbl.loc[uid, label_cols].values.astype(int)

        if {'x1','x2','y1','y2'}.issubset(g.columns):
            xs = ((g['x1'] + g['x2']) / 2.0).values
            ys = ((g['y1'] + g['y2']) / 2.0).values
            w_arr = (g['x2'] - g['x1']).values
            h_arr = (g['y2'] - g['y1']).values
        else:
            xs = g['x'].values
            ys = g['y'].values
            w_arr = g['width'].values
            h_arr = g['height'].values

        if len(xs) >= 5:
            x_c = float(np.median(xs))
            y_lo, y_hi = np.quantile(ys, [0.02, 0.98])
            base_side = float(np.median(np.maximum(w_arr, h_arr)))
        else:
            x_c = img_size / 2.0
            y_lo, y_hi = img_size * 0.1, img_size * 0.9
            base_side = img_size * 0.25

        side = int(max(min_side, base_side * expand))
        for i, v in enumerate(label_cols):
            y_c = float(y_lo + (i + 0.5) * (y_hi - y_lo) / 7.0)
            x1 = int(np.clip(x_c - side / 2, 0, img_size - 1))
            y1 = int(np.clip(y_c - side / 2, 0, img_size - 1))
            x2 = x1 + side
            y2 = y1 + side
            if x2 > img_size: x1 -= (x2 - img_size); x2 = img_size
            if y2 > img_size: y1 -= (y2 - img_size); y2 = img_size
            x1 = max(0, x1); y1 = max(0, y1)
            rows.append(dict(StudyInstanceUID=uid, v=v, v_idx=i, y=int(y7[i]),
                             x1=x1, y1=y1, x2=x2, y2=y2, target=target))
    df = pd.DataFrame(rows)
    if df.empty:
        raise RuntimeError('make_patch_index produced no rows. Check bbox_df content.')
    return df

patch_idx = make_patch_index(train_df, y_df, bbox_df)

class PatchDataset(Dataset):
    def __init__(self, patch_idx, mip_root, transform=None, jitter=0.1):
        self.df = patch_idx.reset_index(drop=True)
        self.root = mip_root
        self.tfm = transform
        self.jitter = jitter
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        arr = np.load(os.path.join(self.root, f'{r.StudyInstanceUID}.npy')).astype(np.float32)[0]
        x1,y1,x2,y2 = int(r.x1),int(r.y1),int(r.x2),int(r.y2)
        # jitter box
        cx = (x1+x2)/2; cy=(y1+y2)/2; side = max(x2-x1, y2-y1)
        cx += np.random.uniform(-side*self.jitter, side*self.jitter)
        cy += np.random.uniform(-side*self.jitter, side*self.jitter)
        scale = np.random.uniform(1-self.jitter, 1+self.jitter)
        side = int(side*scale)
        x1 = int(max(0, cx - side/2)); y1 = int(max(0, cy - side/2))
        x2 = x1 + side; y2 = y1 + side
        x2 = min(x2, arr.shape[1]); y2 = min(y2, arr.shape[0])
        crop = arr[y1:y2, x1:x2]
        crop = cv2.resize(crop, (r.target, r.target), interpolation=cv2.INTER_LINEAR)
        img = np.stack([crop,crop,crop], axis=2)
        if self.tfm: img = self.tfm(image=img)['image']
        y = torch.tensor([r.y], dtype=torch.float32)
        meta = {'uid': r.StudyInstanceUID, 'v_idx': int(r.v_idx)}
        return img, y, meta

train_tfm = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15, rotate_limit=15, p=0.8),
    A.RandomBrightnessContrast(0.15, 0.15, p=0.5),
    A.Normalize(mean=0.5, std=0.5),
    ToTensorV2()
])
val_tfm = A.Compose([A.Normalize(mean=0.5, std=0.5), ToTensorV2()])

def build_model():
    return timm.create_model('tf_efficientnet_b0_ns', pretrained=True, in_chans=3, num_classes=1, drop_rate=0.2, drop_path_rate=0.1).to(device)

mlskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X_ids = train_df[['StudyInstanceUID']]
y7 = X_ids.merge(y_df[['StudyInstanceUID']+label_cols], on='StudyInstanceUID', how='left')[label_cols].values

oof_patch = np.zeros((len(train_df), 7), np.float32)

for fold,(tr,va) in enumerate(mlskf.split(X_ids, y7),1):
    tr_uids = set(X_ids.iloc[tr]['StudyInstanceUID'])
    va_uids = set(X_ids.iloc[va]['StudyInstanceUID'])
    tr_idx = patch_idx[patch_idx['StudyInstanceUID'].isin(tr_uids)]
    va_idx = patch_idx[patch_idx['StudyInstanceUID'].isin(va_uids)]
    tr_ds = PatchDataset(tr_idx, mip_root, train_tfm, jitter=0.2)
    va_ds = PatchDataset(va_idx, mip_root, val_tfm, jitter=0.0)
    # balance positives
    wts = np.where(tr_idx['y'].values==1, 3.0, 1.0).astype(np.float32)
    sampler = WeightedRandomSampler(torch.from_numpy(wts), num_samples=len(wts), replacement=True)
    tr_loader = DataLoader(tr_ds, batch_size=64, sampler=sampler, num_workers=4, pin_memory=True)
    va_loader = DataLoader(va_ds, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

    model = build_model()
    opt = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
    sch = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.5, patience=2, min_lr=1e-6)
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([2.0], device=device))

    best = (1e9, None)
    for epoch in range(8):
        model.train(); tr_loss=0
        for img,y,meta_batch in tr_loader:
            img=img.to(device); y=y.to(device).squeeze(1)
            opt.zero_grad(set_to_none=True)
            logit = model(img).squeeze(1)
            loss = criterion(logit, y)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            tr_loss += loss.item()*img.size(0)
        # val aggregate to study x 7
        model.eval()
        m = {}
        with torch.no_grad():
            for img,y,meta_batch in va_loader:
                logit = model(img.to(device)).squeeze(1)
                p = torch.sigmoid(logit).cpu().numpy()
                uids = meta_batch['uid']
                vidxs = meta_batch['v_idx']
                for j in range(len(p)):
                    uid = uids[j]
                    i = int(vidxs[j].item()) if torch.is_tensor(vidxs[j]) else int(vidxs[j])
                    m.setdefault(uid, {k:[] for k in range(7)})[i].append(float(p[j]))
        va_ids = X_ids.iloc[va]['StudyInstanceUID'].tolist()
        val_mat = np.zeros((len(va_ids),7), np.float32)
        for a,uid in enumerate(va_ids):
            for i in range(7): 
                xs = m.get(uid,{}).get(i,[])
                val_mat[a,i] = np.mean(xs) if xs else 0.0
        # quick early-stop by overall WLL
        y_true = y_df.set_index('StudyInstanceUID').loc[va_ids, label_cols].values
        overall = val_mat.max(1, keepdims=True)
        wll = np.average([log_loss(y_true[:,i], val_mat[:,i], labels=[0,1]) for i in range(7)] + [log_loss(y_true.max(1), overall[:,0], labels=[0,1])], weights=[1]*7+[2])
        sch.step(wll)
        if wll < best[0]: best = (wll, model.state_dict())
        print(f'Fold {fold} Epoch {epoch+1}: Val WLL={wll:.4f}')
    torch.save(best[1], f'patch_fold_{fold}.pth')

    # fill OOF
    model.load_state_dict(best[1]); model.eval()
    m = {}
    with torch.no_grad():
        for img,y,meta_batch in va_loader:
            logit = model(img.to(device)).squeeze(1)
            p = torch.sigmoid(logit).cpu().numpy()
            uids = meta_batch['uid']
            vidxs = meta_batch['v_idx']
            for j in range(len(p)):
                uid = uids[j]
                i = int(vidxs[j].item()) if torch.is_tensor(vidxs[j]) else int(vidxs[j])
                m.setdefault(uid, {k:[] for k in range(7)})[i].append(float(p[j]))
    for uid in X_ids.iloc[va]['StudyInstanceUID']:
        ridx = train_df.index[train_df['StudyInstanceUID']==uid][0]
        arr = np.zeros(7, np.float32)
        for i in range(7):
            xs = m.get(uid, {}).get(i, [])
            arr[i] = np.mean(xs) if xs else 0.0
        oof_patch[ridx] = arr

patch_oof_df = train_df[['StudyInstanceUID']].copy()
patch_oof_df[label_cols] = oof_patch
patch_oof_df['patient_overall'] = oof_patch.max(axis=1)
patch_oof_df.to_csv('oof_patch_probs.csv', index=False)
p_clip = np.clip(oof_patch, 1e-6, 1 - 1e-6)
np.save('oof_logits_patch.npy', np.log(p_clip / (1.0 - p_clip)))

Bounding boxes columns: ['StudyInstanceUID', 'x', 'y', 'width', 'height', 'slice_number']
First few rows:
            StudyInstanceUID      x      y  width  height  slice_number
0  1.2.826.0.1.3680043.12152  177.0  242.0  107.0    96.0           330
1  1.2.826.0.1.3680043.12152  178.0  243.0  106.0    94.0           331
2  1.2.826.0.1.3680043.12152  180.0  244.0  104.0    92.0           332
3  1.2.826.0.1.3680043.12152  181.0  244.0  102.0    91.0           333
4  1.2.826.0.1.3680043.12152  182.0  245.0  101.0    89.0           334
Using x,y,width,height format


/app/.pip-target/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


Fold 1 Epoch 1: Val WLL=2.8165


Fold 1 Epoch 2: Val WLL=2.8482


Fold 1 Epoch 3: Val WLL=2.7971


...

Fold 5 Epoch 4: Val WLL=1.9848


Fold 5 Epoch 5: Val WLL=2.0041


Fold 5 Epoch 6: Val WLL=2.0059


Fold 5 Epoch 7: Val WLL=1.9841


Fold 5 Epoch 8: Val WLL=1.9808
